In [33]:
import numpy as np
import pandas as pd
from characters.sparkle import Sparkle
from characters.qingque import Qingque
from characters.silver_wolf import Silver_Wolf
from characters.fu_xuan import Fu_Xuan
from characters.dhil import DHIL
from utils.arena import Arena
from utils.data_table import Action_Table, Qingque_Probability_Table

In [34]:
dhil_speed = 98

In [35]:
silver_wolf_speed = 161 

In [36]:
sparkle_speed = 160.1

In [37]:
fu_xuan_speed = 134
fu_xuan_technique = True

In [38]:
def initialize_dhil():
  arena = Arena()
  DH =DHIL(arena = arena, Spd = dhil_speed)
  SW = Silver_Wolf(arena = arena, Spd = silver_wolf_speed)
  FX = Fu_Xuan(arena = arena, tech = fu_xuan_technique, Spd = fu_xuan_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = DH, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, DH, Spk, SW, FX

In [39]:
action_dataframe = Action_Table()

In [40]:
def dhil_sparkle(cycles = 20, verbose = False, action_df = action_dataframe):
    arena, dh, spk, sw, fx = initialize_dhil()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [dh, spk, sw, fx]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if fx.action_gauge == 0:
        
        #Fu Xuan's actions
            if verbose:
                print("Fu Xuan takes a turn")
            if arena.sp > 0 and fx.skill_counter < 0:
                action_df.add_row(arena, fx, "Skill", -1)
                ##TODO: Add FX's 12% CR Buff:
                fx.skill(verbose = verbose)
  
            else:
                action_df.add_row(arena, fx, "Basic", 1)
                fx.basic(verbose = verbose)

            fx.update_buffs()
            fx.action_reset()

        elif sw.action_gauge == 0:
        
        #Silver Wolf's actions
            if verbose:
                print("Silver Wolf takes a turn")
            if arena.sp > 0 and sw.skill_counter <= 0:
                action_df.add_row(arena, sw, "Skill", -1)
                sw.skill(verbose = verbose)

            else:
                action_df.add_row(arena, sw, "Basic", 1)
                sw.basic(verbose = verbose)

            sw.update_buffs()
            sw.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()
        
        #Dan Heng IL's actions
        elif dh.action_gauge == 0:
            dh.skill(verbose=verbose)
            action_df.add_row(arena, dh, f"Enhanced Basic Lvl {dh.basic_level}", -(dh.sp_cost))
            dh.enhanced_basic(verbose=verbose)

            if dh.energy >= dh.max_energy:
                action_df.add_row(arena, dh, "Ultimate", 0)
                dh.ult(verbose=verbose)
            dh.action_reset()

        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [41]:
dhil_sparkle()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


In [45]:
pd.set_option("display.max_rows", None)
dh_action_dataframe = action_dataframe.df.loc[action_dataframe.df["Name"] == "Dan Heng IL"]
action_dataframe.df

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
0,Silver Wolf,0.0,62.111801,6,55.00,Skill,-1
1,Sparkle,0.0,62.460962,5,55.00,Skill,-1
2,Dan Heng IL,0.0,62.560962,4,70.00,Enhanced Basic Lvl 3,-3
3,Fu Xuan,0.0,74.626866,1,67.50,Basic,1
4,Silver Wolf,0.0,124.223602,2,55.00,Basic,1
5,Sparkle,0.0,124.921924,3,90.82,Skill,-1
6,Sparkle,0.0,124.921924,2,126.64,Ultimate,4
7,Dan Heng IL,0.0,125.021924,6,110.00,Enhanced Basic Lvl 3,-3
8,Dan Heng IL,0.0,125.021924,3,150.00,Ultimate,0
9,Fu Xuan,0.0,149.253731,3,91.38,Basic,1


In [46]:
dh_action_dataframe

,Name,Cycle,Action Gauge,Current SP,Current Energy,Action,SP Gain
2,Dan Heng IL,0.0,62.560962,4,70.0,Enhanced Basic Lvl 3,-3
7,Dan Heng IL,0.0,125.021924,6,110.0,Enhanced Basic Lvl 3,-3
8,Dan Heng IL,0.0,125.021924,3,150.0,Ultimate,0
12,Dan Heng IL,1.0,187.482886,4,5.0,Enhanced Basic Lvl 3,-1
16,Dan Heng IL,1.0,249.943848,0,45.0,Enhanced Basic Lvl 0,0
21,Dan Heng IL,2.0,312.404809,6,65.0,Enhanced Basic Lvl 3,-3
25,Dan Heng IL,3.0,374.865771,4,105.0,Enhanced Basic Lvl 3,-3
26,Dan Heng IL,3.0,374.865771,1,145.0,Ultimate,0
30,Dan Heng IL,4.0,476.906588,2,5.0,Enhanced Basic Lvl 3,-1
35,Dan Heng IL,4.0,528.026996,4,45.0,Enhanced Basic Lvl 3,-3
